I was unable to find any large dataset which contained user emails of the type I wanted.
Fitting any clustering algorithm on this dataset was producing satisfactory but not good results
hence, I discarded the idea.
however this notebook still contains much of the work which I tried.

In [53]:
import re

import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
import dill

mails_df = pd.read_csv("../data/training_data.csv", sep='~', index_col=0)
imputer = SimpleImputer(strategy='constant', fill_value='')
mails_df = pd.DataFrame(imputer.fit_transform(mails_df), columns=mails_df.columns)
mails_df.iloc[:10]

,id,sender,receiver,subject,body,labels
0,1871339da28c50be,cognizance@iitr.ac.in,"students-notices@iitr.ac.in,staff-notices@iitr...",Cognizance 2023 is Live Now - Join the Excitem...,"Greetings everyone,We are excited to announce ...","UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
1,187129b4701da92f,dosw@iitr.ac.in,students-notices@iitr.ac.in,Opportunity to the Researchers to write techni...,"Dear Students, Following is the link to the ab...","IMPORTANT,CATEGORY_PERSONAL,INBOX"
2,1871298cfa0ea9df,dosw@iitr.ac.in,students-notices@iitr.ac.in,Advisory to reach your Bhawan by 1 am on March...,"Dear Students, As you are aware, there are man...","UNREAD,IMPORTANT,Label_9,CATEGORY_PERSONAL,INBOX"
3,1871203b08c898ff,gensec.cult@iitr.ac.in,students-notices@iitr.ac.in,"Annual Art Exhibition - ""दर्पण"" | Fine Arts | ...","Greetings!Fine Arts Section, IIT Roorkee is re...","IMPORTANT,Label_7,CATEGORY_PERSONAL,INBOX"
4,18711da44ef7c918,ccf@iitr.ac.in,"students-notices@iitr.ac.in,staff-notices@iitr...",CCF stall with fun games and merchandise in AB...,Dear Campus CommunityCommittee for Campus Faun...,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
5,1870f51bfecd3538,channeli.img@iitr.ac.in,aakash_ks@mfs.iitr.ac.in,"IMG Recruitments, 2023","Hi Aakash Kumar Singh, As the recruitment seas...","IMPORTANT,Label_11,CATEGORY_UPDATES,INBOX,Labe..."
6,1870e6c71fa0bdf1,arcurriculum@iitr.ac.in,"faculty-notices@iitr.ac.in,students-notices@ii...","Mar 24, 2023 as Non Teaching Day","All Faculty Members and Students, It is to rem...","Label_16,IMPORTANT,CATEGORY_PERSONAL,INBOX,Lab..."
7,1870e5349df7bb1a,gensec-sports@iitr.ac.in,students-notices@iitr.ac.in,Schedule for Annual Athletics Meet 2022-23,"Dear All, We are glad to inform you that Insti...","IMPORTANT,CATEGORY_PERSONAL,INBOX"
8,1870e4e0a1a6dba6,"adoswsa@iitr.ac.in,adoswsa@iitr.ac.in","students-notices@iitr.ac.in,students-notices@i...",Fwd: Invitation to participate in Model G20 In...,"Dear Students, IIT Hyderabad is organizing ess...","UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
9,1870d5ca067eaaab,gensec.technical@iitr.ac.in,ug-1st-year@iitr.ac.in,MaRS Recruitment Test,Hey freshers! Hope you all attended the recrui...,"UNREAD,IMPORTANT,Label_11,CATEGORY_PERSONAL,IN..."


In [54]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [55]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return processed_text


def preprocess_sender(address):
    address = address.lower()
    address = re.sub('[.]ac[.]in|[.]com', '', address)
    address = re.sub('@|[.]', ' ', address)
    return address

subject_tfidf = TfidfVectorizer(preprocessor=preprocess_text, min_df=0.01)
body_tfidf = TfidfVectorizer(preprocessor=preprocess_text, max_df=0.8, min_df=0.05)
sender_tfidf = TfidfVectorizer(preprocessor=preprocess_sender)

subject_vectors = subject_tfidf.fit_transform(mails_df['subject'])
body_vectors = body_tfidf.fit_transform(mails_df['body'])
sender_vectors = sender_tfidf.fit_transform(mails_df['sender'])

In [56]:
from scipy.sparse import hstack

feature_matrix = hstack((subject_vectors, body_vectors, sender_vectors))

In [57]:
type(feature_matrix)

scipy.sparse._csr.csr_matrix

In [58]:
preprocess_sender('hello@cs.iitr.ac.in')

'hello cs iitr'

In [59]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.json', 'r') as file:
    all_labels = json.load(file)
label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in mails_df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['CATEGORY_FORUMS', 'CATEGORY_PERSONAL', 'CATEGORY_PROMOTIONS', 'CATEGORY_UPDATES', 'IMPORTANT', 'INBOX', 'SENT', 'STARRED', 'UNREAD'] will be ignored
  warnings.warn(


In [60]:
X = feature_matrix
y = labels
print(X.shape, y.shape)

(1198, 601) (1198, 21)


In [61]:
from sklearn.cluster import KMeans

k = 25
model =  KMeans(n_clusters=25, n_init=10, random_state=42)
labels = model.fit_predict(X.toarray())
labels

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


array([2, 1, 1, ..., 9, 4, 2])

In [62]:
mails_df['label'] = pd.Series(labels)

In [63]:
new_df = mails_df.loc[:,['id', 'subject', 'label', 'labels']]

vectorizers = [sender_tfidf, body_tfidf, subject_tfidf]
dill.dump(vectorizers, open('../data/TfidfVectorizers.pkl', 'wb'))

vectorizers = dill.load(open('../data/TfidfVectorizers.pkl', 'rb'))

In [64]:
new_df.reset_index(inplace=True)

In [65]:
new_df.sort_values(by=['label', 'index'], inplace=True)

In [66]:
new_df

,index,id,subject,label,labels
22,22,187086bb3af1c52c,Resurgence to Sanskrit: Vedic Texts to Virtual...,0,"UNREAD,IMPORTANT,Label_10,CATEGORY_PERSONAL,INBOX"
27,27,18704fc4cb05d93b,Resurgence to Sanskrit: Vedic Texts to Virtual...,0,"UNREAD,IMPORTANT,Label_10,CATEGORY_PERSONAL,INBOX"
35,35,187004d1cf48b154,Slides of lecture 1-10,0,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
145,145,186c9fbfb4a3dc86,Reminder of today's IPA DAE - C. V. Raman Lecture,0,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
151,151,186c69fd4b1790f3,Slides of lecture 1-3,0,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
...,...,...,...,...,...
882,882,185297f775d3a74b,Notice : Appointment of Deputy Director as Cha...,24,"IMPORTANT,CATEGORY_PERSONAL"
912,912,18504d41be051997,OM : Appointment of Deputy Director,24,"IMPORTANT,CATEGORY_PERSONAL"
929,929,184f0919ce041cc4,"OM : Appointment of Faculty Coordinator, Cogni...",24,"IMPORTANT,CATEGORY_PERSONAL"
1025,1025,184ae28ce9b7326e,Notice : Constitution of committee to coordina...,24,"IMPORTANT,CATEGORY_PERSONAL"


In [67]:
new_df.loc[new_df['label'] == 1]

,index,id,subject,label,labels
1,1,187129b4701da92f,Opportunity to the Researchers to write techni...,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
2,2,1871298cfa0ea9df,Advisory to reach your Bhawan by 1 am on March...,1,"UNREAD,IMPORTANT,Label_9,CATEGORY_PERSONAL,INBOX"
46,46,186fdde2a7d2e208,Internship for UG and PG Students at Summer Sc...,1,"Label_3281058460844688860,UNREAD,IMPORTANT,CAT..."
231,231,18697135f15f7871,"Invitation for Technical Festival, NIMBUS-2023...",1,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
254,254,18687746df7c1100,Invitation to participate in G-20 Quiz,1,"UNREAD,IMPORTANT,CATEGORY_PERSONAL"
295,295,1866d044e1710be8,Financial support for Siddharth's family who p...,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
343,343,186504366246911e,Financial support for Siddharth's family who p...,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
350,350,1864eb41b02e1423,APPEAL TO STUDENTS - DONATION FOR THE JOSHIMAT...,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
355,355,1864a59e909162d1,Sad demise of a student,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"
494,494,186031dba0e427a5,Re: OM: Appointment of Dean of International R...,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX"


In [68]:
def write_label_names(label_id_series: pd.Series, exc_list=[]) -> pd.Series:
    """
    :param exc_list: a list of label_ids among the default ones which are required
    will convert comma separated label_ids into label_names for a series of strings
    """
    labels_dict = all_labels

    def label_filter(label_id_st):
        label_names = []
        for label_id in label_id_st:
            if label_id == labels_dict[label_id] and label_id not in exc_list:
                label_id_st.remove(label_id)
            else:
                label_names.append(labels_dict[label_id])

        return ','.join(label_names)

    return label_id_series.str.split(',').apply(label_filter)

In [69]:
new_df['label_list'] = write_label_names(new_df['labels'])

In [70]:
for i in range(25):
    print(new_df.loc[new_df['label'] == i].label_list.value_counts())
    print()

                  29
Guest Lectures     3
Name: label_list, dtype: int64

                       34
bhawan notifs           2
jobs and internship     2
Name: label_list, dtype: int64

                                     371
jobs and internship                   25
IMP                                   11
academics,ETE                          9
Class materials,academics              8
bhawan notifs                          4
institute updates                      3
Guest Lectures                         3
Techincal Clubs                        3
OTP                                    2
academics                              2
SCSP                                   2
MTE,academics                          2
Club Recruitments,Techincal Clubs      1
Club Recruitments                      1
MTE,academics,ETE                      1
hackathons                             1
treks                                  1
Class materials                        1
AAO                                  

In [71]:
model.transform(X)

array([[1.96970888, 2.06037224, 1.6994129 , ..., 2.15384361, 2.01185502,
        2.17659083],
       [1.69187923, 0.66626814, 1.3670271 , ..., 1.83314538, 1.68117022,
        1.90337022],
       [1.95339829, 1.34668407, 1.66020272, ..., 2.10264312, 1.95719404,
        2.16637357],
       ...,
       [1.95191866, 2.06038441, 1.71573021, ..., 2.13498789, 1.9835316 ,
        2.18338198],
       [1.97324945, 2.01927085, 1.70605806, ..., 2.12173017, 1.96312576,
        2.15976325],
       [1.6706233 , 1.81338896, 1.37120131, ..., 1.90841433, 1.74124718,
        1.93836313]])

In [72]:
from sklearn.mixture import GaussianMixture

k = 25
model =  GaussianMixture(n_components=25, n_init=10)
labels = model.fit_predict(X.toarray())
labels

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmea

array([ 9,  7,  7, ...,  5, 24,  9], dtype=int64)

In [73]:
model.converged_

True

In [74]:
model.n_iter_

2

In [75]:
mails_df['label'] = pd.Series(labels)
new_df = mails_df.loc[:, ['id', 'subject', 'label', 'labels']]
new_df.reset_index(inplace=True)
new_df.sort_values(by=['label', 'index'], inplace=True)
new_df

,index,id,subject,label,labels
40,40,186fe3bb3af82477,Re: Invited talk on How AI-enabled Technology ...,0,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
42,42,186fe356991dc705,Re: Invited talk on How AI-enabled Technology ...,0,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
43,43,186fe2c41f0b4896,Invited talk on How AI-enabled Technology Is P...,0,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
44,44,186fe2b352f89aec,Webex meeting invitation: Invited talk on How ...,0,"UNREAD,IMPORTANT,CATEGORY_UPDATES,INBOX"
75,75,186ee532d570c56c,Invitation for our first alumni meet (Nano Mee...,0,"UNREAD,IMPORTANT,Label_10,CATEGORY_PERSONAL,INBOX"
...,...,...,...,...,...
1163,1163,184576fbfab869ca,Updates: Your List of Quarantined Emails since...,24,"Label_3628272813817210666,IMPORTANT,CATEGORY_U..."
1169,1169,18455b822c06d210,Updates: Your List of Quarantined Emails since...,24,"Label_3628272813817210666,IMPORTANT,CATEGORY_U..."
1181,1181,184516d3bcc5a613,Updates: Your List of Quarantined Emails since...,24,"Label_3628272813817210666,IMPORTANT,CATEGORY_U..."
1187,1187,1845091dda68ad71,Updates: Your List of Quarantined Emails since...,24,"Label_3628272813817210666,IMPORTANT,CATEGORY_U..."


In [76]:
new_df.loc[new_df['label'] == 1]

,index,id,subject,label,labels
16,16,1870a21d51aa1525,IIT Roorkee improves its Global Rank in Broad ...,1,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
29,29,187035837722b354,Re: ASME IIT Roorkee Student Section goes Recr...,1,"UNREAD,IMPORTANT,Label_11,CATEGORY_PERSONAL,IN..."
37,37,186fe967d84e544f,Re: ASME IIT Roorkee Student Section goes Recr...,1,"UNREAD,IMPORTANT,Label_11,CATEGORY_PERSONAL,IN..."
47,47,186fdda6bd1f1de4,Free Workshops for IIT Roorkee Students in Cog...,1,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
53,53,186f93d25bc1869e,ASME IIT Roorkee Student Section goes Recruiting!,1,"UNREAD,IMPORTANT,Label_11,CATEGORY_PERSONAL,IN..."
66,66,186ef51d5e29f27a,"Finance Club, IIT Roorkee | WorldQuant BRAIN I...",1,"UNREAD,IMPORTANT,CATEGORY_PERSONAL,INBOX"
130,130,186d051e1d7b2962,IIT Roorkee Motorsports Recruitment 2023,1,"IMPORTANT,Label_11,CATEGORY_PERSONAL,INBOX"
140,140,186cab76605637bd,Enact-A-Thon 4.0 || Enactus IIT-Roorkee,1,"IMPORTANT,Label_11,CATEGORY_PERSONAL,INBOX"
154,154,186c5778e52b3aad,IIT Roorkee Motorsports Recruitment 2023,1,"IMPORTANT,Label_11,CATEGORY_PERSONAL,INBOX"
404,404,18630d4e74351fe2,SAE IIT Roorkee Event Zaplin Junker,1,"IMPORTANT,CATEGORY_PERSONAL,INBOX,Label_269289..."


In [77]:
def write_label_names(label_id_series: pd.Series, exc_list=[]) -> pd.Series:
    """
    :param exc_list: a list of label_ids among the default ones which are required
    will convert comma separated label_ids into label_names for a series of strings
    """
    labels_dict = all_labels

    def label_filter(label_id_st):
        label_names = []
        for label_id in label_id_st:
            if label_id == labels_dict[label_id] and label_id not in exc_list:
                label_id_st.remove(label_id)
            else:
                label_names.append(labels_dict[label_id])

        return ','.join(label_names)

    return label_id_series.str.split(',').apply(label_filter)


new_df['label_list'] = write_label_names(new_df['labels'])
for i in range(25):
    print(new_df.loc[new_df['label'] == i].label_list.value_counts())
    print()

                  64
Guest Lectures     2
IMP                1
Name: label_list, dtype: int64

                                     32
Club Recruitments,Techincal Clubs     3
IMP,Techincal Clubs                   1
jobs and internship                   1
IMP                                   1
Name: label_list, dtype: int64

                             111
Class materials,academics      8
jobs and internship            5
bhawan notifs                  4
academics,ETE                  2
academics                      2
IMP                            2
Guest Lectures                 1
MTE,academics,ETE              1
Class materials                1
quarantined                    1
jobs and internship,IMP        1
Name: label_list, dtype: int64

                                     14
Club Recruitments,Techincal Clubs     1
Name: label_list, dtype: int64

academics,ETE                9
MTE,academics                9
Class materials,academics    8
Quiz,academics               7
MTE,acade

It can be clearly inferred from the last output that the clustering is not well for this scenario